# Fine-tuning Embeddings for RAG on Specific Data





In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies



In [ ]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/1

In [ ]:
!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 20.2 MB/s eta 0:00:00


### Provide OpenAI API Key

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


## Task 2: Loading Data



In [ ]:
!mkdir data

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

path = "data/"
text_loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyPDFLoader)
documents = text_loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

In [ ]:
training_documents = text_splitter.split_documents(text_loader.load())

In [ ]:
len(training_documents)

539

In [ ]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [ ]:
total_len = len(training_documents)

# Ensure we do not create an empty validation set
val_start_idx = max(0, total_len - 24)  # Ensure starting index is within range
test_start_idx = max(0, total_len - 12)  # Ensure starting index for test set

# Split dataset
training_split_documents = training_documents[:val_start_idx]
val_split_documents = training_documents[val_start_idx:test_start_idx]
test_split_documents = training_documents[test_start_idx:]

## Task 3: Constructing a Fine-tuning Dataset



In [ ]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

In [ ]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [ ]:
import asyncio
from tqdm import tqdm

async def process_document(document, n_questions):
   questions_generated = await question_generation_chain.ainvoke({"context": document,"n_questions": n_questions})

   doc_questions = {}
   doc_relevant_docs = {}

   for question in questions_generated.content.split("\n"):
    question_id = str(uuid.uuid4())
    doc_questions[question_id] = "".join(question.split(".")[1:]).strip()
    doc_relevant_docs[question_id] = [document.metadata["id"]]

    return doc_questions, doc_relevant_docs

async def create_questions(documents, n_questions):
  tasks = [process_document(doc, n_questions) for doc in documents]

  questions = {}
  relevant_docs = {}

  for task in tqdm(asyncio.as_completed(tasks), total=len(documents), desc="Processing documents"):
    doc_questions, doc_relevant_docs = await task
    questions.update(doc_questions)
    relevant_docs.update(doc_relevant_docs)

  return questions, relevant_docs

In [ ]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)

Processing documents: 100%|██████████| 515/515 [00:15<00:00, 33.69it/s] 


In [ ]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)

Processing documents: 100%|██████████| 12/12 [00:01<00:00,  6.01it/s]


In [ ]:
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

Processing documents: 100%|██████████| 12/12 [00:01<00:00, 10.05it/s]


In [ ]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [ ]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [ ]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

## Task 4: Fine-tuning `snowflake-arctic-embed-l`



In [ ]:
!pip install -qU sentence_transformers datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.


In [ ]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-m"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [ ]:
BATCH_SIZE = 10

Let's move our dataset into the expected format for training.

In [ ]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

In [ ]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

In [ ]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [ ]:
EPOCHS = 10

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
50,No log,No log,0.833333,1.000000,1.000000,1.000000,0.833333,0.333333,0.200000,0.100000,0.833333,1.000000,1.000000,1.000000,0.927577,0.902778,0.902778
52,No log,No log,0.833333,1.000000,1.000000,1.000000,0.833333,0.333333,0.200000,0.100000,0.833333,1.000000,1.000000,1.000000,0.927577,0.902778,0.902778
100,No log,No log,0.916667,1.000000,1.000000,1.000000,0.916667,0.333333,0.200000,0.100000,0.916667,1.000000,1.000000,1.000000,0.958333,0.944444,0.944444
104,No log,No log,0.916667,1.000000,1.000000,1.000000,0.916667,0.333333,0.200000,0.100000,0.916667,1.000000,1.000000,1.000000,0.958333,0.944444,0.944444
150,No log,No log,0.916667,0.916667,1.000000,1.000000,0.916667,0.305556,0.200000,0.100000,0.916667,0.916667,1.000000,1.000000,0.952556,0.937500,0.937500
156,No log,No log,0.916667,0.916667,1.000000,1.000000,0.916667,0.305556,0.200000,0.100000,0.916667,0.916667,1.000000,1.000000,0.952556,0.937500,0.937500
200,No log,No log,0.916667,0.916667,1.000000,1.000000,0.916667,0.305556,0.200000,0.100000,0.916667,0.916667,1.000000,1.000000,0.952556,0.937500,0.937500
208,No log,No log,0.916667,0.916667,1.000000,1.000000,0.916667,0.305556,0.200000,0.100000,0.916667,0.916667,1.000000,1.000000,0.952556,0.937500,0.937500
250,No log,No log,0.916667,0.916667,1.000000,1.000000,0.916667,0.305556,0.200000,0.100000,0.916667,0.916667,1.000000,1.000000,0.952556,0.937500,0.937500
260,No log,No log,0.916667,0.916667,1.000000,1.000000,0.916667,0.305556,0.200000,0.100000,0.916667,0.916667,1.000000,1.000000,0.952556,0.937500,0.937500


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
hf_username = "rprav007"

In [ ]:
model.push_to_hub(f"{hf_username}/snowflake-arctic-embed-m-finetuned-v1")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

'https://huggingface.co/rprav007/snowflake-arctic-embed-m-finetuned-v1/commit/18b33c033e7a99fdca4ae81ca5e9d5d5a3c5dca0'

## Task 5: Use the fine tuned embedding model and check metrcis



In [2]:
!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [3]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.6 MB/s eta 0:00:00
   

In [4]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

Please enter your OpenAI API key!··········


In [5]:
os.environ["TAVILY_API_KEY"] = getpass("TAVILY_API_KEY")

TAVILY_API_KEY··········


In [6]:
os.environ["GOOGLE_API_KEY"] = getpass("GOOGLE_API_KEY")

GOOGLE_API_KEY··········


In [7]:
os.environ["GOOGLE_CSE_ID"] = getpass("GOOGLE_CSE_ID")

GOOGLE_CSE_ID··········


In [8]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00


In [9]:
!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 15.4 MB/s eta 0:00:00


In [43]:
!pip install -qU sentence_transformers datasets pyarrow langchain_google_community arxiv unstructured[pdf]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.4/523.4 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 111.9 MB/s eta 0:00:00
   ━━━━━━

In [12]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

In [44]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.pdf")
docs = loader.load()

In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

534

In [46]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="rprav007/snowflake-arctic-embed-m-finetuned-v1")

Some weights of BertModel were not initialized from the model checkpoint at rprav007/snowflake-arctic-embed-m-finetuned-v1 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
from langchain_qdrant import QdrantVectorStore
from langchain.embeddings import OpenAIEmbeddings
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="obecity_rag",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="obecity_rag",
    embedding=embeddings,
)

In [48]:
_ = vector_store.add_documents(documents=split_documents)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [49]:
!pip install -qU cohere langchain_cohere

In [19]:
os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

Please enter your Cohere API key!··········


In [50]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [51]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who will do the following:
1. Be clear and detailed
2. Stay relevant to the context of the question

Follow these guidelines while responding:
- Assist in setting realistic and achievable weight-loss goals that are tailored to individual [needs] and [lifestyle]. The process should involve an initial assessment of current habits, health status, and lifestyle to establish a baseline. From there, develop a structured, step-by-step plan that includes short-term milestones and long-term objectives. The plan should be flexible enough to adjust as progress is made but structured enough to provide clear direction. Incorporate strategies for overcoming common obstacles, such as motivation dips and plateaus, and recommend tools or resources for tracking progress. Ensure the goals are SMART (Specific, Measurable, Achievable, Relevant, and Time-bound) to increase the likelihood of success.
- Your task is to identify and help address unhelpful eating patterns in the client seeking to improve their health and wellness. Begin by conducting a comprehensive assessment to understand the client's current eating habits, lifestyle, and underlying factors contributing to their eating patterns. Develop a personalized plan that incorporates achievable goals, mindful eating strategies, and healthier food choices. Provide ongoing support, motivation, and adjustments to the plan based on the client’s progress and feedback. Your approach should be empathetic, evidence-based, and tailored to each client's unique needs, aiming to foster sustainable, positive changes in their eating habits.
- Act as a fitness coach. Develop a personalized workout routine specifically tailored to meet the client's [fitness goal]. The routine must consider the client's current fitness level, any potential limitations or injuries, and their available equipment. It should include a mix of cardiovascular exercises, strength training, flexibility workouts, and recovery activities. Provide clear instructions for each exercise, suggest the number of sets and repetitions, and offer guidance on proper form to maximize effectiveness and minimize the risk of injury.
- As a Personal Chef specialized in creating customized meal plans, design a meal plan tailored to specific dietary preferences. This plan should cater to the client's [health goals], [taste preferences], and any [dietary restrictions] they might have. The meal plan should cover breakfast, lunch, dinner, and snack options for one week, ensuring a balanced and nutritious diet. Include a detailed list of ingredients for each meal, preparation instructions that are easy to follow, and tips for meal prepping to save time.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [52]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [53]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [54]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [55]:
graph_builder = StateGraph(State).add_sequence([retrieve_adjusted, generate])
graph_builder.add_edge(START, "retrieve_adjusted")
graph = graph_builder.compile()

In [56]:
response = graph.invoke({"question" : "Why is obesity a big problem in America?"})

In [57]:
response["response"]

'Obesity is a significant public health issue in America, and its prevalence can be attributed to a combination of complex factors including lifestyle, environment, genetics, and socio-economic determinants. Currently, about **42% of adults in the United States** are classified as obese, with a BMI of **30 or greater**. This has far-reaching implications, both for individual health and the broader healthcare system. Here are several key reasons why obesity is a major problem in America:\n\n### 1. **Health Risks**\n   - **Increased Comorbidities**: Obesity is associated with a higher risk of numerous health conditions including type 2 diabetes, hypertension, cardiovascular diseases, and sleep disorders. These comorbidities can lead to significant health complications and reduce quality of life.\n   - **Premature Mortality**: The link between obesity and early death is well established, illustrating how excessive body weight can lead to life-threatening health issues.\n\n### 2. **Economi

In [58]:
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage


def ai_rag_tool(question: str) -> str:
    """Useful for when you need to answer questions about obesity. Input should be a fully formed question."""
    response = graph.invoke({"question" : question})
    return {
        "messages" : [HumanMessage(content=response["response"])],
        "context" : response["context"]
    }

ai_rag_tool_instance = Tool(
    name="Obesity_QA_Tool",  # ✅ No spaces, only letters, numbers, underscores, or hyphens
    description="Useful for when you need to answer questions about obesity. Input should be a fully formed question.",
    func=ai_rag_tool
)

In [59]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import Tool

tavily_tool = TavilySearchResults(max_results=5)
google_search = Tool(
    name="GoogleSearch",
    func=GoogleSearchAPIWrapper().run, # Use the .run method directly
    description="Use this tool to search Google.", # Provide a description
)

def tavily_search_func(query: str):
    return TavilySearchResults(max_results=5).invoke({"query": query})

tavily_tool_instance = Tool(
    name="TavilySearch",
    func=tavily_search_func,  # ✅ Now has a proper function name
    description="Use this tool to search Tavily."
)

def arxiv_query_func(query: str):
    return ArxivQueryRun().invoke({"query": query})

arxiv_tool_instance = Tool(
    name="ArxivQuery",
    func=arxiv_query_func,  # ✅ Named function
    description="Use this tool to search academic papers on Arxiv."
)

tool_belt = [
    tavily_tool_instance,  # ✅ Now a valid `Tool`
    arxiv_tool_instance,  # ✅ Now a valid `Tool`
    google_search,  # ✅ Already correct
    ai_rag_tool_instance,  # ✅ Already correct
]

In [60]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

In [61]:
model = model.bind_tools(tool_belt)

In [62]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.documents import Document

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]
  context: List[Document]

In [63]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {
        "messages" : [response],
        "context" : state.get("context", [])
  }

tool_node = ToolNode(tool_belt)

In [64]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

In [65]:
uncompiled_graph.set_entry_point("agent")

In [66]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

In [67]:
uncompiled_graph.add_edge("action", "agent")

In [68]:
compiled_graph = uncompiled_graph.compile()

In [69]:
from langchain_core.messages import HumanMessage

#inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}
inputs = {"messages" : [HumanMessage(content="What is the impact of weightloss on obesity. You will perform perform evidence search using Arxiv and correlate with statements from the news article that needs to be verified using Tavily, and match them with reliable sources such as government or healthcare websites using google searchthat corroborate the findings.")]}

#inputs = {"messages" : [HumanMessage(content="Who is the current captain of winnipeg jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iKpNAAMcE2b4jI5KB5YPTQp6', 'function': {'arguments': '{"__arg1": "impact of weight loss on obesity"}', 'name': 'ArxivQuery'}, 'type': 'function'}, {'id': 'call_biHTRXeZfmVPAkLEsmXY1h7I', 'function': {'arguments': '{"__arg1": "impact of weight loss on obesity"}', 'name': 'TavilySearch'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 215, 'total_tokens': 278, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_eb9dce56a8', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2941cc55-0b52-40c7-8c40-000928bbcb04-0', tool_calls=[{'name': 'ArxivQuery', 'args': {'__arg1': 'impact of weight loss on obes

In [70]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [71]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/4 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/4 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/85 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/98 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [72]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Who Jacinda M. Nicklas be in obesity research?,[ClinicalReview&Education JAMA | Review Obesit...,"Jacinda M. Nicklas, MD, MPH, is one of the aut...",single_hop_specifc_query_synthesizer
1,What gonna happen with obesity rates in US by ...,[Epidemiology The prevalence of obesity worldw...,"By 2030, it is anticipated that 48.9% of US ad...",single_hop_specifc_query_synthesizer
2,Cud yu explane the role of ghrelin in the path...,[Pathophysiology of Obesity Influenced by gene...,Ghrelin is one of the hormones involved in the...,single_hop_specifc_query_synthesizer
3,How does the World Health Organization classif...,[Diagnosis and Classification of Obesity Body ...,The World Health Organization uses BMI to defi...,single_hop_specifc_query_synthesizer
4,How do digital health interventions enhance ob...,[<1-hop>\n\nAbstract The U.S. Preventive Servi...,Digital health interventions enhance obesity s...,multi_hop_abstract_query_synthesizer
5,How do global obesity trends and obesity manag...,[<1-hop>\n\nClinicalReview&Education JAMA | Re...,Global obesity trends have shown a significant...,multi_hop_abstract_query_synthesizer
6,How do global obesity trends and obesity manag...,[<1-hop>\n\nClinicalReview&Education JAMA | Re...,Global obesity trends indicate a significant i...,multi_hop_abstract_query_synthesizer
7,How have recent advances in understanding the ...,[<1-hop>\n\nDownloaded from https://www.scienc...,Recent advances in understanding the molecular...,multi_hop_abstract_query_synthesizer
8,How do the interactions between short- and lon...,[<1-hop>\n\nof body weight (fat mass) to the s...,The interactions between short- and long-term ...,multi_hop_specific_query_synthesizer
9,How does the combination of Phentermine-Topira...,[<1-hop>\n\nt s e v i t c e p s o r t e r d n ...,The combination of Phentermine-Topiramate cont...,multi_hop_specific_query_synthesizer


In [73]:
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage

def parse_context_from_invoked_tools(invoked_tools_log: list, tool_belt: list, query: str) -> dict:
    context_list = []
    tool_dict = {tool.name: tool for tool in tool_belt if hasattr(tool, "name")}

    for tool_name in invoked_tools_log:
        if tool_name in tool_dict:
            tool = tool_dict[tool_name]
            try:
                response = tool.func(query)
                if isinstance(response, dict):
                    extracted_text = "\n".join(msg.content for msg in response.get("messages", []) if hasattr(msg, "content"))
                    context_list.append(extracted_text)
                else:
                    context_list.append(str(response))
            except Exception as e:
                context_list.append(f"Error extracting context from {tool.name}: {str(e)}")

    # 🔥 Ensure at least one valid update
    if not context_list:
        context_list.append("No relevant context retrieved from tools.")

    final_context = "\n".join(context_list)

    return {
        "messages": [HumanMessage(content=final_context)],
        "context": final_context  # Always a string
    }


async def extract_invoked_tools_context(compiled_graph, inputs, tool_belt):
    invoked_tools_log = []  # Track invoked tools
    formated_inputs = {"messages": [HumanMessage(content=inputs["question"])]}

    async for chunk in compiled_graph.astream(formated_inputs, stream_mode="updates"):
        for node, values in chunk.items():
            print(f"Receiving update from node: '{node}'")

            if node == "action":
                tool_used = values["messages"][0].name
                print(f"✅ Tool Used: {tool_used}")
                invoked_tools_log.append(tool_used)

            #print(f"📥 Messages Received: {values['messages']}\n")

    # Remove duplicates and parse context
    invoked_tools_log = list(set(invoked_tools_log))
    query = inputs.get("question", "")
    dynamic_context = parse_context_from_invoked_tools(invoked_tools_log, tool_belt, query)

    # 🔥 Debug Final Context
    #print(f"🛠️ Final Extracted Context: {dynamic_context}")

    return dynamic_context


In [74]:
for test_row in dataset:
    user_query = test_row.eval_sample.user_input
    inputs = {"question": user_query}
    print (inputs)

        # Step 1: Extract dynamic context
    dynamic_context = await extract_invoked_tools_context(compiled_graph, inputs, tool_belt)
    #print("Dynamic Context Extracted:", dynamic_context)

    #    Step 2: Invoke the graph
    response = graph.invoke(inputs)

        # Step 3: Debugging Step - Check response
    #print("Graph Response:", response)

        # Step 4: Inject responses and context
    test_row.eval_sample.response = response.get("response", "No response")

        # Ensure retrieved_contexts is properly structured
    if "context" in response and response["context"]:
        test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
    elif "context" in dynamic_context:
        test_row.eval_sample.retrieved_contexts = [dynamic_context["context"]]
    else:
        test_row.eval_sample.retrieved_contexts = ["No context available."]

{'question': 'Who Jacinda M. Nicklas be in obesity research?'}
Receiving update from node: 'agent'
Receiving update from node: 'action'
✅ Tool Used: Obesity_QA_Tool
Receiving update from node: 'agent'
{'question': 'What gonna happen with obesity rates in US by 2030 and what WHO doing about it?'}
Receiving update from node: 'agent'
Receiving update from node: 'action'
✅ Tool Used: Obesity_QA_Tool
Receiving update from node: 'agent'
{'question': 'Cud yu explane the role of ghrelin in the pathophysiology of obesity?'}
Receiving update from node: 'agent'
Receiving update from node: 'action'
✅ Tool Used: Obesity_QA_Tool
Receiving update from node: 'agent'
{'question': 'How does the World Health Organization classify obesity using BMI?'}
Receiving update from node: 'agent'
Receiving update from node: 'action'
✅ Tool Used: Obesity_QA_Tool
Receiving update from node: 'agent'
{'question': 'How do digital health interventions enhance obesity screening and treatment, and what challenges do they f

In [75]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Who Jacinda M. Nicklas be in obesity research?,[INTRODUCTION The last decade has ushered in a...,[ClinicalReview&Education JAMA | Review Obesit...,Jacinda M. Nicklas is likely to be a significa...,"Jacinda M. Nicklas, MD, MPH, is one of the aut...",single_hop_specifc_query_synthesizer
1,What gonna happen with obesity rates in US by ...,[Epidemiology\n\nThe prevalence of obesity wor...,[Epidemiology The prevalence of obesity worldw...,### Obesity Rates in the U.S. by 2030\n\nAccor...,"By 2030, it is anticipated that 48.9% of US ad...",single_hop_specifc_query_synthesizer
2,Cud yu explane the role of ghrelin in the path...,[Pathophysiology of Obesity\n\nInfluenced by g...,[Pathophysiology of Obesity Influenced by gene...,**The Role of Ghrelin in the Pathophysiology o...,Ghrelin is one of the hormones involved in the...,single_hop_specifc_query_synthesizer
3,How does the World Health Organization classif...,"[Body mass index, calculated as weight in kilo...",[Diagnosis and Classification of Obesity Body ...,The World Health Organization (WHO) classifies...,The World Health Organization uses BMI to defi...,single_hop_specifc_query_synthesizer
4,How do digital health interventions enhance ob...,[Digitally-Delivered Weight Management Interve...,[<1-hop>\n\nAbstract The U.S. Preventive Servi...,Digital health interventions (DHIs) significan...,Digital health interventions enhance obesity s...,multi_hop_abstract_query_synthesizer
5,How do global obesity trends and obesity manag...,[INTRODUCTION The last decade has ushered in a...,[<1-hop>\n\nClinicalReview&Education JAMA | Re...,The intersection of global obesity trends and ...,Global obesity trends have shown a significant...,multi_hop_abstract_query_synthesizer
6,How do global obesity trends and obesity manag...,[Epidemiology\n\nThe prevalence of obesity wor...,[<1-hop>\n\nClinicalReview&Education JAMA | Re...,Global obesity trends and obesity management s...,Global obesity trends indicate a significant i...,multi_hop_abstract_query_synthesizer
7,How have recent advances in understanding the ...,[INTRODUCTION The last decade has ushered in a...,[<1-hop>\n\nDownloaded from https://www.scienc...,Recent advances in understanding the molecular...,Recent advances in understanding the molecular...,multi_hop_abstract_query_synthesizer
8,How do the interactions between short- and lon...,[INTERACTING SHORT- AND LONG-TERM SYSTEMS REGU...,[<1-hop>\n\nof body weight (fat mass) to the s...,The effectiveness of leptin therapy in treatin...,The interactions between short- and long-term ...,multi_hop_specific_query_synthesizer
9,How does the combination of Phentermine-Topira...,"[obesity (249, 251). Two doses (7.5/46 and 15/...",[<1-hop>\n\nt s e v i t c e p s o r t e r d n ...,The combination of phentermine and topiramate ...,The combination of Phentermine-Topiramate cont...,multi_hop_specific_query_synthesizer


## Re-evaluate the metrics to compare with original

In [76]:
from ragas import EvaluationDataset
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[17]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[29]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[35]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[41]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[47]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[53]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[59]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[65]: TimeoutError()
ERROR:ragas.executor:Exception raised in Job[71]: TimeoutError()


{'context_recall': 0.7305, 'faithfulness': 0.4999, 'factual_correctness': 0.4200, 'answer_relevancy': 0.8779, 'context_entity_recall': 0.3830, 'noise_sensitivity_relevant': 0.1860}

## Compare Results before and after embedding fine tuning

**context_recall**:               *Before*: 0.6563,     *After*: 0.7305

**faithfulness**:                 *Before*: 0.5725,     *After*: 0.4999

**factual_correctness**:        *Before*: 0.3325,           *After*: 0.4200

**answer_relevancy**:           *Before*: 0.9476, *After*: 0.8779

**context_entity_recall**:      *Before*: 0.2558, *After*: 0.3830

**noise_sensitivity_relevant**: *Before*: 0.2241,     *After*: 0.1860